In [ ]:
pip install mediapipe opencv-python


Restrição de Território com controle via teclado 
Nome: Gabriel Cesar Vendramini 
NºUSP: 14571272

In [ ]:
import cv2
import mediapipe as mp

# Inicializa o detector de poses e desenhista
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Defina a área limitada inicial
LIMITED_AREA = [(100, 100), (500, 500)]  # Exemplo: retângulo com vértices (100, 100) e (500, 500)

def is_within_area(x, y, area):
   #Verifica se um ponto (x, y) está dentro da área limitada.
    (x_min, y_min), (x_max, y_max) = area
    return x_min <= x <= x_max and y_min <= y <= y_max

# Carregar um arquivo de vídeo
cap = cv2.VideoCapture(0)  # Altere o caminho para o seu vídeo

# Verifica se o vídeo foi carregado corretamente
if not cap.isOpened():
    print("Erro ao abrir o vídeo")
    exit()

# Obtém a resolução original do vídeo
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Redimensionar o frame para garantir as dimensões corretas
        frame = cv2.resize(frame, (frame_width, frame_height))

        # Converter a imagem para RGB
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        
        # Desenhar a área limitada
        cv2.rectangle(frame, LIMITED_AREA[0], LIMITED_AREA[1], (0, 255, 0), 2)

        # Se forem detectadas poses
        if results.pose_landmarks:
            # Desenhar os pontos do esqueleto
            mp_drawing.draw_landmarks(
                frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
            # Verificar se os pontos do esqueleto estão dentro da área limitada
            for landmark in results.pose_landmarks.landmark:
                x = int(landmark.x * frame.shape[1])
                y = int(landmark.y * frame.shape[0])
                
                # Verifica se cada ponto está dentro da área limitada
                if is_within_area(x, y, LIMITED_AREA):
                    cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)  # Ponto dentro da área
                else:
                    cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)  # Ponto fora da área
        
        # Mostrar a imagem com a área e a pose desenhada
        cv2.imshow('Skeleton Detection', frame)
        
        # Ler a tecla pressionada
        key = cv2.waitKey(10) & 0xFF
        
        # Controle do retângulo
        if key == ord('w'):  # Move o retângulo para cima
            LIMITED_AREA = [(LIMITED_AREA[0][0], LIMITED_AREA[0][1] - 10),
                            (LIMITED_AREA[1][0], LIMITED_AREA[1][1] - 10)]
        elif key == ord('s'):  # Move o retângulo para baixo
            LIMITED_AREA = [(LIMITED_AREA[0][0], LIMITED_AREA[0][1] + 10),
                            (LIMITED_AREA[1][0], LIMITED_AREA[1][1] + 10)]
        elif key == ord('a'):  # Move o retângulo para a esquerda
            LIMITED_AREA = [(LIMITED_AREA[0][0] - 10, LIMITED_AREA[0][1]),
                            (LIMITED_AREA[1][0] - 10, LIMITED_AREA[1][1])]
        elif key == ord('d'):  # Move o retângulo para a direita
            LIMITED_AREA = [(LIMITED_AREA[0][0] + 10, LIMITED_AREA[0][1]),
                            (LIMITED_AREA[1][0] + 10, LIMITED_AREA[1][1])]
        elif key == ord('q'):  # Aumenta o tamanho do retângulo
            LIMITED_AREA = [(LIMITED_AREA[0][0] - 5, LIMITED_AREA[0][1] - 5),
                            (LIMITED_AREA[1][0] + 5, LIMITED_AREA[1][1] + 5)]
        elif key == ord('e'):  # Diminui o tamanho do retângulo
            LIMITED_AREA = [(LIMITED_AREA[0][0] + 5, LIMITED_AREA[0][1] + 5),
                            (LIMITED_AREA[1][0] - 5, LIMITED_AREA[1][1] - 5)]
        
        # Novos comandos para ajustar altura e largura
        elif key == ord('u'):  # Aumenta a altura do retângulo
            LIMITED_AREA = [(LIMITED_AREA[0][0], LIMITED_AREA[0][1] - 5),
                            (LIMITED_AREA[1][0], LIMITED_AREA[1][1] + 5)]
        elif key == ord('i'):  # Diminui a altura do retângulo
            LIMITED_AREA = [(LIMITED_AREA[0][0], LIMITED_AREA[0][1] + 5),
                            (LIMITED_AREA[1][0], LIMITED_AREA[1][1] - 5)]
        elif key == ord('j'):  # Aumenta a largura do retângulo
            LIMITED_AREA = [(LIMITED_AREA[0][0] - 5, LIMITED_AREA[0][1]),
                            (LIMITED_AREA[1][0] + 5, LIMITED_AREA[1][1])]
        elif key == ord('k'):  # Diminui a largura do retângulo
            LIMITED_AREA = [(LIMITED_AREA[0][0] + 5, LIMITED_AREA[0][1]),
                            (LIMITED_AREA[1][0] - 5, LIMITED_AREA[1][1])]
        # código para sair do loop pela tecla Esc
        elif key == 27:  # Tecla Esc
            break


cap.release()
cv2.destroyAllWindows()
